# Import packages and data

In [1]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import pickle
from helper_functions import *
from helpers_lanes import *
from Vehicles import *
from Line import *

with open('svc.p', 'rb') as f:
    dist_pickle = pickle.load(f)
    svc = dist_pickle["svc"]
    X_scaler = dist_pickle["scaler"]
    orient = dist_pickle["orient"]
    pix_per_cell = dist_pickle["pix_per_cell"]
    cell_per_block = dist_pickle["cell_per_block"]
    spatial_size = dist_pickle["spatial_size"]
    hist_bins = dist_pickle["hist_bins"]
    
with open('cam_cal.p', 'rb') as f:
    dist_pickle = pickle.load(f)
    mtx = dist_pickle["mtx"]
    dist = dist_pickle["dist"]
    del dist_pickle

ystarts = [400, 400, 400, 400]
ystops = [496, 528, 592, 656]
scales = [1., 1.25, 1.5, 1.75]

# Define pipeline function

In [2]:
def combo_pipeline(img, mtx, dist, left, right,
                  cars, ystarts, ystops, scales,
                  svc, X_scaler, orient, pix_per_cell,
                  cell_per_block, spatial_size, hist_bins):
    
    img0 = undistort(img, mtx, dist)
    
    # Detect lane lines
    img, Minv = warp(img0)
    img = combine_threds(img)
    
    if left.detected and right.detected:
        left_x, left_y, right_x, right_y = locate_lanes_skip_window(
            img, left.current_fit, right.current_fit)
        left_fitparam, right_fitparam = fit_poly(left_x, left_y, right_x, right_y)
        left_fit_m, right_fit_m = fit_poly_m(left_x, left_y, right_x, right_y)
        _, ploty, left_xfit, right_xfit = fit_poly_plot(
            img, left_fitparam, right_fitparam, left_x, left_y, right_x, right_y)
        left_curv, right_curv = get_curv_m(img, left_fit_m, right_fit_m)
        left_dist, right_dist = dist2center_m(img, left_fitparam, right_fitparam)

        if validate_lane(
            left_curv, right_curv, left_dist, right_dist, left_fitparam, right_fitparam):
            left.update_line(left_x, left_y, left_fitparam, left_xfit, left_curv, left_dist)
            right.update_line(right_x, right_y, right_fitparam, right_xfit, right_curv, right_dist)
        else:
            left.detected = False
            right.detected = False

    else:
        left_x, left_y, right_x, right_y = locate_lanes(img)
        left_fitparam, right_fitparam = fit_poly(left_x, left_y, right_x, right_y)
        left_fit_m, right_fit_m = fit_poly_m(left_x, left_y, right_x, right_y)
        _, ploty, left_xfit, right_xfit = fit_poly_plot(
            img, left_fitparam, right_fitparam, left_x, left_y, right_x, right_y)
        left_curv, right_curv = get_curv_m(img, left_fit_m, right_fit_m)
        left_dist, right_dist = dist2center_m(img, left_fitparam, right_fitparam)

        if validate_lane(left_curv, right_curv, left_dist, right_dist, left_fitparam, right_fitparam):
            left.update_line(left_x, left_y, left_fitparam, left_xfit, left_curv, left_dist)
            right.update_line(right_x, right_y, right_fitparam, right_xfit, right_curv, right_dist)
        else:
            left.detected = False
            right.detected = False
    
    # Detect vehicles
    box_list = []
    cars.img = img0
    if (cars.frame % 12 == 0):
        for ystart, ystop, scale in zip(ystarts, ystops, scales):
            _, boxes = find_cars(img0, ystart, ystop, scale,
                         svc, X_scaler, orient, pix_per_cell,
                         cell_per_block, spatial_size, hist_bins)
            box_list.extend(boxes)
        car_bbox = apply_heat(img0, box_list)
        draw_img = cars.draw_over_frames(car_bbox)

    elif (cars.frame % 6 == 0) and (cars.new_ystop > 0):
        new_ystops = [cars.new_ystop if x>cars.new_ystop else x for x in ystops]
        k = len(scales)
        for ystart, ystop, scale in zip(ystarts, new_ystops, scales):
            _, boxes = find_cars(img0, ystart, ystop, scale,
                         svc, X_scaler, orient, pix_per_cell,
                         cell_per_block, spatial_size, hist_bins)
            box_list.extend(boxes)
        car_bbox = apply_heat(img0, box_list)
        draw_img = cars.draw_over_frames(car_bbox)

    else:
        draw_img = draw_boxes(img0, cars.current_bboxes)

    cars.frame += 1
    
    # Project lane lines
    result = project_lines(draw_img, img, Minv, ploty, left.bestx, right.bestx)
    curverad = (left.radius_of_curvature + right.radius_of_curvature) / 2
    if left.line_base_pos < right.line_base_pos:
        str_side = 'left'
    else:
        str_side = 'right'
    dist2center = np.absolute((right.line_base_pos - left.line_base_pos) / 2.)

    cv2.putText(result,
                'Radius of Curvature = {:.0f}m'.format(curverad),
                (100, 50),
                fontFace=cv2.FONT_HERSHEY_PLAIN,
                fontScale = 4,
                color=(255,255,255),
                thickness=2,)
    cv2.putText(result,
                'Vehicle is {:.2f}m {:s} of center'.format(dist2center, str_side),
                (100, 100),
                fontFace=cv2.FONT_HERSHEY_PLAIN,
                fontScale = 4,
                color=(255,255,255),
                thickness=2,)
    return result

def process_image(img):
    return combo_pipeline(img, mtx, dist, left, right,
                  cars, ystarts, ystops, scales,
                  svc, X_scaler, orient, pix_per_cell,
                  cell_per_block, spatial_size, hist_bins)

# Process video

In [3]:
cars = Vehicles()
left = Line()
right = Line()
video_output = 'combined_output.mp4'
clip1 = VideoFileClip('project_video.mp4')
video_clip = clip1.fl_image(process_image)
%time video_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video combined_output.mp4
[MoviePy] Writing video combined_output.mp4


100%|█████████▉| 1260/1261 [07:17<00:00,  1.82it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: combined_output.mp4 

CPU times: user 6min 56s, sys: 54.4 s, total: 7min 51s
Wall time: 7min 18s


In [4]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))